In [121]:
import numpy as np

def LeastSquare(regressor, response):
    # use numpy.linalg.lstsq
    # modify residual and singular value, eliminate the effect caused by data size
    if regressor.shape[0] <= regressor.shape[1]:
        raise ValueError("regressor size error, #datums <= #variables")
    
    coe, res, rank, singular_value = np.linalg.lstsq(regressor, response, rcond=-1)
    rms = res / (regressor.shape[0] - regressor.shape[1]) # residual mean square
    return coe, rms, rank, singular_value

def PCStd(data):
    # data : numpy matrix
    # return standard deviation of each principal component
    if data.shape[0] < 2:
        raise ValueError("data size error, variance of single datum is meaningless")
    
    data -= data.mean(axis=0)
    output = np.linalg.eigh(np.dot(data.T, data))[0][::-1]/(data.shape[0] - 1)
    output = np.sqrt(output)
    return output

class LinearRegression():
    def __init__(self, regressor, response, has_bias = True, regularizer = 10**-10):
        self.SetData(regressor, response, has_bias)
        self.SetRegularizer(regularizer)
    
    def SetData(self, regressor, response, has_bias):
        if (regressor.ndim != 2) or (regressor.size == 0):
            raise ValueError("regressor should be a non-empty numpy matrix")
        elif (response.ndim != 2) or (response.size == 0):
            raise ValueError("response should be a non-empty numpy matrix")
        elif len(regressor) != len(response):
            raise ValueError("len(regressor) != len(response)")
        
        self.num_var = regressor.shape[1]
        self.has_bias = has_bias
        if self.has_bias:
            self.regressor = np.append(regressor, np.ones((len(regressor), 1)), axis=1)
        else:
            self.regressor = regressor
        
        self.response = response
    
    def SetRegularizer(self, regularizer):
        if regularizer < 0.:
            self.regularizer = 0.
            print("SetRegularizer error, regularizer must be non-negative, has been set to zero.")
        else:
            self.regularizer = regularizer
    
    def MostUselesscAnalysis(self, drop=1):
        gram = np.dot(self.regressor.T, self.regressor)
        gram += self.regularizer * self.regressor.shape[0] * np.identity(self.regressor.shape[1])
        projected = np.dot(self.regressor.T, self.response)
        is_leave = np.ones((gram.shape[0]), dtype=bool)
        response_square = np.square(self.response).sum(axis=0)
        for d in range(drop):
            square_err = np.inf * np.ones((gram.shape[0]), dtype=bool)
            for i in range(self.num_var):
                if is_leave[i]:
                    work_index = (is_leave * (np.arange(len(is_leave)) != i)).astype(bool)
                    coe = np.linalg.solve(gram[work_index][:, work_index], projected[work_index])
                    square_err[i] = (response_square
                                     - (coe * projected[work_index]).sum(axis=0) 
                                     - self.regularizer * np.square(coe).sum(axis=0)
                                    ).sum()
            
            is_leave[np.argmin(square_err)] = False
        
        rms = square_err.min()/(self.regressor.shape[0] - is_leave.sum())
        coe = np.linalg.solve(gram[is_leave][:, is_leave], projected[is_leave])
        
        if self.has_bias:
            return is_leave[:-1], coe[:-1], coe[-1].reshape(1, -1), rms
        else:
            return is_leave, coe, np.zeros((1, self.num_var)), rms